In [1]:
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
%load_ext autoreload
%autoreload 2

In [3]:
from generate_robot_instruction import encode_task_generation_prompt, generate_task_data, encode_instruct_prompt, generate_instruction_following_chat_data, post_process_chat_response

### Generate Tasks

In [7]:
generate_task_data(
    output_dir="./gpt4_generation/task_generation/",
    num_tasks_to_generate=100,
    model_name="gpt-4-0314"
)

### [[{'role': 'user', 'content': "You are serving as a task-generation helper for a given robot environment.\xa0\n\n[Environment Description]\nHere is the information about the environment. The environment is called {Two Block World}\xa0\nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on the table.\nContents in the {} are the names of the objects in the environment.\n\nThe observation of the robot includes:\nThe position and orientation of both blocks and the end effector of the robot\nThe force sensor on the end effector. \n\n[Your Task]\nCome up with 50 different tasks for the robot to perform.\n\n[Output format]\nThe response should follow the template below:\n### Task {i}: {task description}\nwhere {i} is the task number and



prompts: 100%|██████████| 1/1 [01:04<00:00, 64.31s/it]


['', ' Task 1: Pick up block A and stack it on top of block B\n', ' Task 2: Pick up block B and stack it on top of block A\n', ' Task 3: Identify and pick up the heavier block*\n', ' Task 4: Identify and pick up the lighter block*\n', ' Task 5: Move block A to the opposite corner of the table\n', ' Task 6: Move block B to the opposite corner of the table\n', ' Task 7: Place both blocks side by side with a gap of 5 cm between them\n', ' Task 8: Stack block A and block B horizontally, with block A on the left and block B on the right\n', ' Task 9: Stack block B and block A horizontally, with block B on the left and block A on the right\n', ' Task 10: Transfer block A from one corner of the table to another without knocking over block B\n', ' Task 11: Transfer block B from one corner of the table to another without knocking over block A\n', ' Task 12: Move block B to a distance of 15 cm away from block A\n', ' Task 13: Move block A to a distance of 15 cm away from block B\n', ' Task 14: S



100%|██████████| 100/100 [02:02<00:00,  1.23s/it]

['', ' Task 1: Pick up block A and place it on top of block B.\n', ' Task 2: Pick up block B and place it on top of block A.\n', ' Task 3: Lift block A and hold it in the air for 5 seconds.*\n', ' Task 4: Lift block B and hold it in the air for 5 seconds.*\n', ' Task 5: Move block A to the left edge of the table.\n', ' Task 6: Move block B to the right edge of the table.\n', ' Task 7: Move the heavier block to the corner of the table.*\n', ' Task 8: Move the lighter block to the corner of the table.*\n', ' Task 9: Stack the blocks horizontally.*\n', ' Task 10: Stack the blocks vertically.*\n', ' Task 11: Move block A close to block B within 5cm.\n', ' Task 12: Move block B close to block A within 5cm.\n', ' Task 13: Rotate block A 180 degrees.\n', ' Task 14: Rotate block B 180 degrees.\n', ' Task 15: Move block A to the center of the table.\n', ' Task 16: Move block B to the center of the table.\n', ' Task 17: Maximize the combined height of the two blocks.*\n', ' Task 18: Minimize the

#### Split task data
Split it into tasks used by instruction generation.

In [10]:
import re, os
import string
import json
def split_json_file(json_file, 
                    output_path="./gpt4_generation/task_generation/", n=2):
  """Splits a JSON file into N JSONL files, where each file contains n elements.

  Args:
    json_file: The path to the JSON file to split.
    n: The number of JSONL files to create.

  Returns:
    The list of paths to the created JSONL files.
  """

  with open(json_file, "r") as f:
    data = json.load(f)
  os.makedirs(output_path, exist_ok=True)
  jsonl_files = []
  for i in range(0, len(data), n):
    jsonl_file = f"{output_path}split_{i // n}.jsonl"
    with open(jsonl_file, "w") as outfile:
      for record in data[i : i + n]:
        json.dump(record, outfile)
        outfile.write("\n")

    jsonl_files.append(jsonl_file)
  return jsonl_files

In [11]:
jsonl_files = split_json_file("./gpt4_generation/task_generation/task_regen.json", n=50)

### Generate COT instruct data

In [12]:
from generate_robot_instruction import generate_instruction_following_chat_data


ret = generate_instruction_following_chat_data(output_dir="./gpt4_generation/cot_full",
                                        instruction_prompt_file="./prompts_for_gpt/robot_cot_prompts.txt",
                                        seed_example_path="./prompts_for_gpt/seeded_example_cot.jsonl",
                                        seed_tasks_path="./gpt4_generation/task_generation/split_0.jsonl",
                                        num_instructions_to_generate=250,
                                        output_file="cot_instrut_regen.json"
                                        )

### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on the table.\nThe observation space is a 26 dimension vector, consisting of:\nblock A position(3), block A orientation(4), bl



prompts: 100%|██████████| 1/1 [02:20<00:00, 140.47s/it]


  4%|▍         | 10/250 [02:20<9:21:52, 140.47s/it]
Request 1 took 140.47s
Generated 10 instructions, kept 10 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on 



prompts: 100%|██████████| 1/1 [01:03<00:00, 63.49s/it]


  6%|▌         | 15/250 [03:23<59:13, 15.12s/it]
Request 2 took 63.49s
Generated 5 instructions, kept 5 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on the ta

...

 97%|█████████▋| 243/250 [1:05:11<02:02, 17.51s/it]
Request 39 took 92.22s
Generated 5 instructions, kept 5 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on th



prompts: 100%|██████████| 1/1 [00:49<00:00, 49.86s/it]


 98%|█████████▊| 246/250 [1:06:00<01:00, 15.25s/it]
Request 40 took 49.87s
Generated 3 instructions, kept 3 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on th



prompts: 100%|██████████| 1/1 [01:33<00:00, 93.79s/it]
253it [1:07:34, 16.03s/it]                         

253it [1:07:34, 18.53s/it]
Request 41 took 93.79s
Generated 7 instructions, kept 7 instructions


In [13]:
from generate_robot_instruction import generate_instruction_following_chat_data


ret = generate_instruction_following_chat_data(output_dir="./gpt4_generation/cot_full",
                                        instruction_prompt_file="./prompts_for_gpt/robot_cot_prompts.txt",
                                        seed_example_path="./prompts_for_gpt/seeded_example_cot.jsonl",
                                        seed_tasks_path="./gpt4_generation/task_generation/split_1.jsonl",
                                        num_instructions_to_generate=250,
                                        output_file="cot_instrut_regen_2.json"
                                        )

### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on the table.\nThe observation space is a 26 dimension vector, consisting of:\nblock A position(3), block A orientation(4), bl



prompts: 100%|██████████| 1/1 [03:19<00:00, 199.85s/it]


  4%|▍         | 10/250 [03:19<13:19:23, 199.85s/it]
Request 1 took 199.85s
Generated 10 instructions, kept 10 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on



prompts: 100%|██████████| 1/1 [00:56<00:00, 56.24s/it]


  6%|▌         | 14/250 [04:16<1:12:06, 18.33s/it]
Request 2 took 56.25s
Generated 4 instructions, kept 4 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on the 

...

 96%|█████████▌| 240/250 [1:11:33<02:36, 15.65s/it]
Request 39 took 88.22s
Generated 4 instructions, kept 4 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on th



prompts: 100%|██████████| 1/1 [02:01<00:00, 121.92s/it]


 98%|█████████▊| 246/250 [1:13:35<01:16, 19.04s/it]
Request 40 took 121.92s
Generated 6 instructions, kept 6 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to come up with a set of task instructions and corresponding responses for each of them.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on t



100%|██████████| 250/250 [1:15:05<00:00, 18.02s/it]

100%|██████████| 250/250 [1:15:05<00:00, 17.74s/it]
Request 41 took 90.51s
Generated 4 instructions, kept 4 instructions


In [48]:
import utils
utils.old_inst_json_to_gorilla_form_json('./gpt4_generation_cot/cot_instrut_regen.json', './gpt4_generation_cot/cot_instrut_regen_gorilla.json')
utils.json2jsonl('./gpt4_generation_cot/cot_instrut_regen_gorilla.json', './gpt4_generation_cot/cot_instrut_regen_gorilla.jsonl')

In [51]:
data_lists = list(open("./gpt4_generation_cot/cot_instrut_regen_gorilla.jsonl"))

In [52]:
data = data_lists[0]
data = json.loads(data)['code']
if '###Instruction.' in data:
    data = data.replace('###Instruction.', '###Instruction:')
if 'Output:' in data:
    a = {'instruction':data.split('Output:')[0].split('Instruction:')[1].split('###')[0],
        'input': data.split('Output:')[0].split('Input:')[1].split('###')[0],
        'output': data.split('Output:')[1]}

In [61]:
data.split('Output:')[1]

" {'explanation': No. We don't know the weight of the blocks, need to infer weight first., 'code': <nooutput>.}\n"